# Customer Lifetime Value (CLTV) Prediction

#### CLTV Prediction with BG-NBD and Gamma-Gamma

CLTV is the monetary value that a customer will bring to the company during the relationship and communication with a company.

CLTV prediction: <br>
Probabilistic lifetime value estimation with time projection <br>
CLTV = Expected sales x Expected Average Profit <br>
CLTV = BG-NBD Model X Gamma Gamma <br>

#### Business Problem:

The UK-based retail company wants to set a roadmap for its sales and marketing activities. In order for the company to make a medium-long-term plan, it is necessary to estimate the potential value that existing customers will provide to the company in the future.

Data can be download from: https://archive.ics.uci.edu/ml/datasets/Online+Retail+II

The dataset named Online Retail II includes online sales transactions of a UK-based retail company between 01/12/2010 and 09/12/2011. The company's product catalog includes souvenirs and it is known that most of its customers are wholesalers.

InvoiceNo: Invoice Number (If this code starts with C, it means that the transaction has been cancelled) <br>
StockCode: Product code (unique for each product) <br>
Description: Product name <br>
Quantity: Number of products (How many of the products on the invoices were sold) <br>
InvoiceDate: Invoice date <br>
UnitPrice: Invoice price <br>
CustomerID: Unique customer number <br>
country: country name <br>

### 1. Importing the Libraries and Uploading Dataset

In [1]:
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
from lifetimes.plotting import plot_period_transactions

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', lambda x: '%.4f' % x)
from sklearn.preprocessing import MinMaxScaler

In [2]:
df_ = pd.read_excel("online_retail.xlsx", sheet_name="Year 2010-2011")

In [3]:
df = df_.copy()
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.5500,17850.0000,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.3900,17850.0000,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.7500,17850.0000,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.3900,17850.0000,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.3900,17850.0000,United Kingdom


### 2. General Information About Dataset

In [4]:
    print("###############################    Shape  ##################################")
    print(df.shape)
    print("###############################    Types  ##################################")
    print(df.dtypes)
    print("###############################    Head   ##################################")
    print(df.head())
    print("###############################    Tail   ##################################")
    print(df.tail())
    print("###############################    NA     ##################################")
    print(df.isnull().sum())
    print("############################### Quantiles ##################################")
    print(df.describe([0, 0.05, 0.50, 0.95, 0.99, 1]).T)

###############################    Shape  ##################################
(541910, 8)
###############################    Types  ##################################
Invoice                object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
Price                 float64
Customer ID           float64
Country                object
dtype: object
###############################    Head   ##################################
  Invoice StockCode                          Description  Quantity         InvoiceDate  Price  Customer ID         Country
0  536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6 2010-12-01 08:26:00 2.5500   17850.0000  United Kingdom
1  536365     71053                  WHITE METAL LANTERN         6 2010-12-01 08:26:00 3.3900   17850.0000  United Kingdom
2  536365    84406B       CREAM CUPID HEARTS COAT HANGER         8 2010-12-01 08:26:00 2.7500   17850.0000  United Kingdom
3  536365  

#### 2.1 Let's remove the missing observations from the dataset:

NA amount:

Description      1454

Customer ID    135080

In [5]:
df.dropna(inplace=True)
df.isnull().sum()

Invoice        0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
Price          0
Customer ID    0
Country        0
dtype: int64

#### 2.2 The 'C' in the invoices shows the canceled transactions. Remove the canceled transactions from the dataset

In [6]:
df[df["Invoice"].str.contains("C", na=False)].head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
141,C536379,D,Discount,-1,2010-12-01 09:41:00,27.5000,14527.0000,United Kingdom
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.6500,15311.0000,United Kingdom
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,2010-12-01 10:24:00,1.6500,17548.0000,United Kingdom
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.2900,17548.0000,United Kingdom
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.2900,17548.0000,United Kingdom


In [7]:
df = df[~df["Invoice"].str.contains("C", na=False)] # removing canceled transactions

In [8]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,397925.0000,13.0218,180.4200,1.0000,2.0000,6.0000,12.0000,80995.0000
Price,397925.0000,3.1162,22.0968,0.0000,1.2500,1.9500,3.7500,8142.7500
Customer ID,397925.0000,15294.3086,1713.1727,12346.0000,13969.0000,15159.0000,16795.0000,18287.0000


### 3. To get rid of the outliers, let's first prepare our functions

In [9]:
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01) #quantile value can be change
    quartile3 = dataframe[variable].quantile(0.99) #quantile value can be change
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit


def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    # dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit # we do not need for our dataset
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit

In [10]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,397925.0000,13.0218,180.4200,1.0000,2.0000,6.0000,12.0000,80995.0000
Price,397925.0000,3.1162,22.0968,0.0000,1.2500,1.9500,3.7500,8142.7500
Customer ID,397925.0000,15294.3086,1713.1727,12346.0000,13969.0000,15159.0000,16795.0000,18287.0000


In [11]:
replace_with_thresholds(df, "Quantity")
replace_with_thresholds(df, "Price")
df.describe().T # max value has changed dramatically

,count,mean,std,min,25%,50%,75%,max
Quantity,397925.0000,11.8337,25.5345,1.0000,2.0000,6.0000,12.0000,298.5000
Price,397925.0000,2.8932,3.2271,0.0000,1.2500,1.9500,3.7500,37.0600
Customer ID,397925.0000,15294.3086,1713.1727,12346.0000,13969.0000,15159.0000,16795.0000,18287.0000


#### 3.1 Creating a variable named 'TotalPrice' that represents the total earnings per invoice

In [12]:
df["TotalPrice"] = df["Quantity"] * df["Price"]
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6.0000,2010-12-01 08:26:00,2.5500,17850.0000,United Kingdom,15.3000
1,536365,71053,WHITE METAL LANTERN,6.0000,2010-12-01 08:26:00,3.3900,17850.0000,United Kingdom,20.3400
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8.0000,2010-12-01 08:26:00,2.7500,17850.0000,United Kingdom,22.0000
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6.0000,2010-12-01 08:26:00,3.3900,17850.0000,United Kingdom,20.3400
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6.0000,2010-12-01 08:26:00,3.3900,17850.0000,United Kingdom,20.3400


In [13]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,397925.0000,11.8337,25.5345,1.0000,2.0000,6.0000,12.0000,298.5000
Price,397925.0000,2.8932,3.2271,0.0000,1.2500,1.9500,3.7500,37.0600
Customer ID,397925.0000,15294.3086,1713.1727,12346.0000,13969.0000,15159.0000,16795.0000,18287.0000
TotalPrice,397925.0000,20.6277,51.8263,0.0000,4.6800,11.8000,19.8000,3268.5750


### 4. Preparation of Lifetime Data Structure

CLTV = ( Customer Value / Churn Rate ) x Profit Margin<br>
Customer Value = Average Order Value * Purchase Frequency<br>
Average Order Value = Total Price / Total transaction<br>
Purchase Frequency = Total Transaction / Total Number of Customers<br>
Churn Rate = 1 - Repeat Rate<br>
Repeat Rate = Number of customers making multiple purchases / All customers<br>
Profit Margin = Total Price * 0.10<br>

Recency: How many weeks was the customer when he made the last purchase (Last Purchase Date - First Purchase Date)<br>
T: The age of the customer. (Analysis Date - First Purchase Date)<br>
Frequency. How often did the customer shop? (Must be greater than one)<br>
Monetary: Average earnings per purchase<br>

In [14]:
today_date = df["InvoiceDate"].max() + dt.timedelta(days=2) # analyse date

In [15]:
cltv_df = df.groupby('Customer ID').agg(
    {'InvoiceDate': [lambda InvoiceDate: (InvoiceDate.max() - InvoiceDate.min()).days,
                     lambda InvoiceDate: (today_date - InvoiceDate.min()).days],
     'Invoice': lambda Invoice: Invoice.nunique(),
     'TotalPrice': lambda TotalPrice: TotalPrice.sum(),
     'Country': lambda Country: Country.min()}) # no needed (country) for CLTV but but I will use in my analysis
cltv_df.head()

InvoiceDate             Invoice TotalPrice         Country
             <lambda_0> <lambda_1> <lambda>   <lambda>        <lambda>
Customer ID                                                           
12346.0000            0        327        1   310.4400  United Kingdom
12347.0000          365        368        7  4310.0000         Iceland
12348.0000          282        359        4  1770.7800         Finland
12349.0000            0         20        1  1491.7200           Italy
12350.0000            0        311        1   331.4600          Norway

In [16]:
cltv_df.columns = cltv_df.columns.droplevel(0)
cltv_df.head()

,<lambda_0>,<lambda_1>,<lambda>,<lambda>,<lambda>
Customer ID,,,,,
12346.0000,0,327,1,310.4400,United Kingdom
12347.0000,365,368,7,4310.0000,Iceland
12348.0000,282,359,4,1770.7800,Finland
12349.0000,0,20,1,1491.7200,Italy
12350.0000,0,311,1,331.4600,Norway


#### 4.1 Let's change the names of the metrics we created to recency, T, frequency and monetary

In [17]:
cltv_df.columns = ['recency', 'T', 'frequency', 'monetary','Country']

In [18]:
cltv_df["monetary"] = cltv_df["monetary"] / cltv_df["frequency"]
cltv_df.describe().T

,count,mean,std,min,25%,50%,75%,max
recency,4339.0000,130.4185,132.0392,0.0000,0.0000,92.0000,251.5000,373.0000
T,4339.0000,224.2600,117.9157,2.0000,114.0000,250.0000,328.0000,375.0000
frequency,4339.0000,4.2720,7.7055,1.0000,1.0000,2.0000,5.0000,210.0000
monetary,4339.0000,364.0132,367.1614,0.0000,176.7664,288.1800,421.9862,6207.6700


In [19]:
cltv_df = cltv_df[(cltv_df['frequency'] > 1)]
cltv_df = cltv_df[(cltv_df['recency'] > 0)]

In [20]:
cltv_df["recency"] = cltv_df["recency"] / 7 # we will do weekly analysis

cltv_df["T"] = cltv_df["T"] / 7
cltv_df.head()

,recency,T,frequency,monetary,Country
Customer ID,,,,,
12347.0000,52.1429,52.5714,7,615.7143,Iceland
12348.0000,40.2857,51.2857,4,442.6950,Finland
12352.0000,37.1429,42.5714,8,219.5425,Norway
12356.0000,43.1429,46.7143,3,937.1433,Portugal
12358.0000,21.2857,21.7143,2,575.2100,Austria


In [21]:
cltv_df.describe().T

,count,mean,std,min,25%,50%,75%,max
recency,2783.0000,29.0481,15.8779,0.1429,15.1429,30.1429,43.0714,53.2857
T,2783.0000,37.4334,14.4568,1.5714,28.5714,40.7143,51.2857,53.5714
frequency,2783.0000,6.0744,9.1367,2.0000,2.0000,4.0000,7.0000,210.0000
monetary,2783.0000,372.3314,338.9085,3.4500,193.3753,302.7900,437.4695,5724.3026


### 5. Building of BG-NBD Model

Let's model it for "United Kingdom" customers

In [22]:
cltv_df_eng = cltv_df[(cltv_df['Country'].str.contains("United Kingdom", na=False))]

In [23]:
bgf = BetaGeoFitter(penalizer_coef=0.001)

bgf.fit(cltv_df_eng['frequency'],
        cltv_df_eng['recency'],
        cltv_df_eng['T'])

<lifetimes.BetaGeoFitter: fitted with 2516 subjects, a: 0.09, alpha: 12.30, b: 2.43, r: 2.25>

#### 5.1 Let's find the 10 customers we expect to purchase the most in 6 months

In [24]:
bgf.predict(4*6,
            cltv_df_eng['frequency'],
            cltv_df_eng['recency'],
            cltv_df_eng['T']).sort_values(ascending=False).head(10)

Customer ID
12748.0000   76.0195
17841.0000   45.3168
13089.0000   35.8149
14606.0000   34.1960
15311.0000   33.4057
12971.0000   31.7091
13408.0000   22.9917
18102.0000   22.6276
13798.0000   21.2981
14527.0000   20.7046
dtype: float64

In [25]:
cltv_df_eng["expected_purc_6_month"] = bgf.predict(4 * 6,
                                               cltv_df_eng['frequency'],
                                               cltv_df_eng['recency'],
                                               cltv_df_eng['T'])
cltv_df.describe().T

,count,mean,std,min,25%,50%,75%,max
recency,2783.0000,29.0481,15.8779,0.1429,15.1429,30.1429,43.0714,53.2857
T,2783.0000,37.4334,14.4568,1.5714,28.5714,40.7143,51.2857,53.5714
frequency,2783.0000,6.0744,9.1367,2.0000,2.0000,4.0000,7.0000,210.0000
monetary,2783.0000,372.3314,338.9085,3.4500,193.3753,302.7900,437.4695,5724.3026


### 6. Building of GAMMA-GAMMA Model

In [26]:
ggf = GammaGammaFitter(penalizer_coef=0.01)

ggf.fit(cltv_df_eng['frequency'], cltv_df_eng['monetary'])

<lifetimes.GammaGammaFitter: fitted with 2516 subjects, p: 3.81, q: 0.35, v: 3.75>

#### 6.1 Let's find the 10 customers we expected profit the most

In [27]:
ggf.conditional_expected_average_profit(cltv_df_eng['frequency'],
                                        cltv_df_eng['monetary']).sort_values(ascending=False).head(10)

Customer ID
14088.0000   3916.4763
18102.0000   3870.8585
15749.0000   3213.6206
14096.0000   3196.0298
17511.0000   2950.3755
17450.0000   2874.0646
13081.0000   2617.2446
16984.0000   2452.7327
16684.0000   2223.7143
16253.0000   2016.6153
dtype: float64

In [28]:
cltv_df_eng["expected_average_profit"] = ggf.conditional_expected_average_profit(cltv_df_eng['frequency'],
                                                                             cltv_df_eng['monetary'])
cltv_df_eng.describe().T

,count,mean,std,min,25%,50%,75%,max
recency,2516.0000,29.1621,15.8641,0.1429,15.4286,30.4286,43.0000,53.2857
T,2516.0000,37.5384,14.4822,1.5714,29.0000,40.8571,51.4286,53.5714
frequency,2516.0000,6.0322,8.4841,2.0000,2.0000,4.0000,7.0000,210.0000
monetary,2516.0000,347.1669,279.9074,3.4500,188.8713,292.1194,417.3249,3864.5546
expected_purc_6_month,2516.0000,3.6734,3.2225,0.0000,2.0331,3.0089,4.4181,76.0195
expected_average_profit,2516.0000,365.4170,290.0341,5.8234,200.0496,307.2486,438.2124,3916.4763


In [29]:
cltv_df_eng.sort_values("expected_average_profit", ascending=False).head(10)

,recency,T,frequency,monetary,Country,expected_purc_6_month,expected_average_profit
Customer ID,,,,,,,
14088.0000,44.5714,46.1429,13,3864.5546,United Kingdom,6.0939,3916.4763
18102.0000,52.2857,52.5714,60,3859.7391,United Kingdom,22.6276,3870.8585
15749.0000,13.8571,47.5714,3,3028.7800,United Kingdom,0.7810,3213.6206
14096.0000,13.8571,14.7143,17,3163.5882,United Kingdom,16.3808,3196.0298
17511.0000,52.8571,53.5714,31,2933.9431,United Kingdom,11.8789,2950.3755
17450.0000,51.2857,52.7143,46,2863.2749,United Kingdom,17.4345,2874.0646
13081.0000,51.2857,53.1429,11,2576.1255,United Kingdom,4.7305,2617.2446
16984.0000,5.8571,18.7143,2,2240.6750,United Kingdom,2.5122,2452.7327
16684.0000,50.4286,51.2857,28,2209.9691,United Kingdom,11.1825,2223.7143


### 7. Calculation of CLTV with BG-NBD and GG model

In [30]:
cltv_eng = ggf.customer_lifetime_value(bgf,
                                   cltv_df_eng['frequency'],
                                   cltv_df_eng['recency'],
                                   cltv_df_eng['T'],
                                   cltv_df_eng['monetary'],
                                   time=6,  # 6 aylık
                                   freq="W",  # T'nin frekans bilgisi.
                                   discount_rate=0.01)
cltv_eng = cltv_eng.reset_index()
cltv_final_eng = cltv_df_eng.merge(cltv_eng, on="Customer ID", how="left")
cltv_final_eng.sort_values(by="clv", ascending=False).head(10)

,Customer ID,recency,T,frequency,monetary,Country,expected_purc_6_month,expected_average_profit,clv
2433,18102.0000,52.2857,52.5714,60,3859.7391,United Kingdom,22.6276,3870.8585,91802.8907
587,14096.0000,13.8571,14.7143,17,3163.5882,United Kingdom,16.3808,3196.0298,54820.9804
2142,17450.0000,51.2857,52.7143,46,2863.2749,United Kingdom,17.4345,2874.0646,52519.0298
2170,17511.0000,52.8571,53.5714,31,2933.9431,United Kingdom,11.8789,2950.3755,36733.8048
1776,16684.0000,50.4286,51.2857,28,2209.9691,United Kingdom,11.1825,2223.7143,26062.6373
405,13694.0000,52.7143,53.5714,50,1275.7005,United Kingdom,18.6780,1280.1635,25061.7502
585,14088.0000,44.5714,46.1429,13,3864.5546,United Kingdom,6.0939,3916.4763,25012.0851
1162,15311.0000,53.2857,53.5714,91,667.7791,United Kingdom,33.4057,669.0804,23427.0545
133,13089.0000,52.2857,53.0000,97,606.3625,United Kingdom,35.8149,607.4745,22803.7870
1047,15061.0000,52.5714,53.2857,48,1120.6019,United Kingdom,18.0460,1124.6958,21273.0610


### 8. Creating Segments by CLTV

In [31]:
cltv_final_eng["segment"] = pd.qcut(cltv_final_eng["clv"], 4, labels=["D", "C", "B", "A"])
cltv_final_eng.sort_values(by="clv", ascending=False)

,Customer ID,recency,T,frequency,monetary,Country,expected_purc_6_month,expected_average_profit,clv,segment
2433,18102.0000,52.2857,52.5714,60,3859.7391,United Kingdom,22.6276,3870.8585,91802.8907,A
587,14096.0000,13.8571,14.7143,17,3163.5882,United Kingdom,16.3808,3196.0298,54820.9804,A
2142,17450.0000,51.2857,52.7143,46,2863.2749,United Kingdom,17.4345,2874.0646,52519.0298,A
2170,17511.0000,52.8571,53.5714,31,2933.9431,United Kingdom,11.8789,2950.3755,36733.8048,A
1776,16684.0000,50.4286,51.2857,28,2209.9691,United Kingdom,11.1825,2223.7143,26062.6373,A
...,...,...,...,...,...,...,...,...,...,...
1419,15881.0000,7.4286,52.1429,5,100.2780,United Kingdom,0.0334,104.6136,3.6663,D
2359,17912.0000,7.7143,52.2857,5,62.2020,United Kingdom,0.0364,65.1862,2.4870,D
1800,16725.0000,9.8571,53.1429,7,111.5743,United Kingdom,0.0132,114.9236,1.5935,D
1068,15107.0000,8.7143,53.2857,6,53.2500,United Kingdom,0.0194,55.4589,1.1271,D


In [32]:
cltv_final_eng.groupby("segment").agg({"count", "mean", "sum"})

Customer ID                         recency                        T                    frequency               monetary                      expected_purc_6_month                  expected_average_profit                        clv                       
              count          sum       mean   count        sum    mean count        sum    mean     count   sum    mean    count         sum     mean                 count       sum   mean                   count         sum     mean count          sum      mean
segment                                                                                                                                                                                                                                                               
D               629 9867051.0000 15686.8855     629 14851.8571 23.6119   629 25826.5714 41.0597       629  1959  3.1145      629 110793.9259 176.1430                   629 1126.9737 1.7917                     629 119941.4454 190.6859   629  187200.6172  297.6162
C               629 9757667.0000 15512.9841     629 19502.8571 31.0061   629 24194.0000 38.4642       629  2574  4.0922      629 166068.2992 264.0196                   629 1780.1483 2.8301                     629 177046.6279 281.4732   629  460526.7598  732.1570
B               629 9798118.0000 15577.2941     629 19050.5714 30.2871   629 22441.2857 35.6777       629  3466  5.5103      629 224293.7285 356.5878                   629 2353.3649 3.7414                     629 235988.5553 375.1805   629  814163.3572 1294.3774
A               629 9665769.0000 15366.8824     629 19966.5714 31.7434   629 21984.7143 34.9519       629  7178 11.4118      629 372316.0677 591.9174                   629 3981.7713 6.3303                     629 386412.6302 614.3285   629 2403892.6635 3821.7689

### 9. Functionalization of the Study

In [33]:
def create_cltv_p(dataframe, month=3): # parameters are: dataframe and analyse time period 
    # 1. Data Preprocessing
    dataframe.dropna(inplace=True)
    dataframe = dataframe[~dataframe["Invoice"].str.contains("C", na=False)]
    dataframe = dataframe[dataframe["Quantity"] > 0]
    dataframe = dataframe[dataframe["Price"] > 0]
    replace_with_thresholds(dataframe, "Quantity")
    replace_with_thresholds(dataframe, "Price")
    dataframe["TotalPrice"] = dataframe["Quantity"] * dataframe["Price"]
    today_date = df["InvoiceDate"].max() + dt.timedelta(days=2)

    cltv_df = dataframe.groupby('Customer ID').agg(
        {'InvoiceDate': [lambda InvoiceDate: (InvoiceDate.max() - InvoiceDate.min()).days,
                         lambda InvoiceDate: (today_date - InvoiceDate.min()).days],
         'Invoice': lambda Invoice: Invoice.nunique(),
         'TotalPrice': lambda TotalPrice: TotalPrice.sum()})

    cltv_df.columns = cltv_df.columns.droplevel(0)
    cltv_df.columns = ['recency', 'T', 'frequency', 'monetary']
    cltv_df["monetary"] = cltv_df["monetary"] / cltv_df["frequency"]
    cltv_df = cltv_df[(cltv_df['frequency'] > 1)]
    cltv_df["recency"] = cltv_df["recency"] / 7
    cltv_df["T"] = cltv_df["T"] / 7

    # 2. Building of BG-NBD Model
    bgf = BetaGeoFitter(penalizer_coef=0.001)
    bgf.fit(cltv_df['frequency'],
            cltv_df['recency'],
            cltv_df['T'])

    cltv_df["expected_purc_1_week"] = bgf.predict(1,
                                                  cltv_df['frequency'],
                                                  cltv_df['recency'],
                                                  cltv_df['T'])

    cltv_df["expected_purc_1_month"] = bgf.predict(4,
                                                   cltv_df['frequency'],
                                                   cltv_df['recency'],
                                                   cltv_df['T'])

    cltv_df["expected_purc_3_month"] = bgf.predict(12,
                                                   cltv_df['frequency'],
                                                   cltv_df['recency'],
                                                   cltv_df['T'])

    # 3. Building of GAMMA-GAMMA Model
    ggf = GammaGammaFitter(penalizer_coef=0.01)
    ggf.fit(cltv_df['frequency'], cltv_df['monetary'])
    cltv_df["expected_average_profit"] = ggf.conditional_expected_average_profit(cltv_df['frequency'],
                                                                                 cltv_df['monetary'])

    # 4. Calculation of CLTV with BG-NBD and GG Model
    cltv = ggf.customer_lifetime_value(bgf,
                                       cltv_df['frequency'],
                                       cltv_df['recency'],
                                       cltv_df['T'],
                                       cltv_df['monetary'],
                                       time=month,  # 3 aylık
                                       freq="W",  # T'nin frekans bilgisi.
                                       discount_rate=0.01)

    cltv = cltv.reset_index()
    cltv_final = cltv_df.merge(cltv, on="Customer ID", how="left")
    cltv_final["segment"] = pd.qcut(cltv_final["clv"], 4, labels=["D", "C", "B", "A"])

    return cltv_final

### 9.1 Let's calculate 1-month and 12-month CLTV for UK customers

In [34]:
df_eng = df_.copy()

In [35]:
df_eng = df_eng[(df_eng['Country'].str.contains("United Kingdom", na=False))]
df_eng.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.5500,17850.0000,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.3900,17850.0000,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.7500,17850.0000,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.3900,17850.0000,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.3900,17850.0000,United Kingdom


In [36]:
cltv_final_eng1 = create_cltv_p(df_eng,month = 1) # for 1-month
cltv_final_eng1

,Customer ID,recency,T,frequency,monetary,expected_purc_1_week,expected_purc_1_month,expected_purc_3_month,expected_average_profit,clv,segment
0,12747.0000,52.2857,52.8571,11,381.4555,0.2024,0.8073,2.4048,387.8230,336.6136,A
1,12748.0000,53.1429,53.5714,209,154.5642,3.2262,12.8703,38.3518,154.7086,2140.8190,A
2,12749.0000,29.8571,30.5714,5,814.4880,0.1670,0.6653,1.9746,844.0953,603.6352,A
3,12820.0000,46.1429,46.8571,4,235.5850,0.1036,0.4132,1.2298,247.0812,109.7700,C
4,12822.0000,2.2857,12.5714,2,474.4400,0.1288,0.5113,1.5070,520.8292,286.1894,B
...,...,...,...,...,...,...,...,...,...,...,...
2565,18272.0000,34.8571,35.4286,6,513.0967,0.1711,0.6819,2.0262,528.7864,387.6293,A
2566,18273.0000,36.4286,37.0000,3,68.0000,0.1041,0.4146,1.2321,73.4340,32.7336,D
2567,18282.0000,16.8571,18.1429,2,89.0250,0.1355,0.5385,1.5916,99.3948,57.5271,D
2568,18283.0000,47.5714,48.2857,16,130.9300,0.3009,1.1999,3.5729,132.5808,171.0360,B


In [37]:
cltv_final_eng12 = create_cltv_p(df_eng,month = 12) # for 12-month
cltv_final_eng12

,Customer ID,recency,T,frequency,monetary,expected_purc_1_week,expected_purc_1_month,expected_purc_3_month,expected_average_profit,clv,segment
0,12747.0000,52.2857,52.8571,11,381.4555,0.2024,0.8073,2.4048,387.8230,3695.1345,A
1,12748.0000,53.1429,53.5714,209,154.5642,3.2262,12.8703,38.3518,154.7086,23533.1293,A
2,12749.0000,29.8571,30.5714,5,814.4880,0.1670,0.6653,1.9746,844.0953,6530.6955,A
3,12820.0000,46.1429,46.8571,4,235.5850,0.1036,0.4132,1.2298,247.0812,1200.0411,C
4,12822.0000,2.2857,12.5714,2,474.4400,0.1288,0.5113,1.5070,520.8292,3019.8419,B
...,...,...,...,...,...,...,...,...,...,...,...
2565,18272.0000,34.8571,35.4286,6,513.0967,0.1711,0.6819,2.0262,528.7864,4210.9734,A
2566,18273.0000,36.4286,37.0000,3,68.0000,0.1041,0.4146,1.2321,73.4340,355.5692,D
2567,18282.0000,16.8571,18.1429,2,89.0250,0.1355,0.5385,1.5916,99.3948,612.6744,D
2568,18283.0000,47.5714,48.2857,16,130.9300,0.3009,1.1999,3.5729,132.5808,1874.2255,B


In [38]:
cltv_final_eng1.groupby("segment").agg({"count", "mean","median", "sum"})

Customer ID                                recency                                T                          frequency                     monetary                            expected_purc_1_week                       expected_purc_1_month                       expected_purc_3_month                        expected_average_profit                                 clv                           
             median count           sum       mean  median count        sum    mean  median count        sum    mean    median count   sum    mean   median count         sum     mean               median count      sum   mean                median count      sum   mean                median count       sum   mean                  median count         sum     mean   median count         sum     mean
segment                                                                                                                                                                                                                                                                                                                                                                                                          
D        15756.0000   643 10100185.0000 15707.9082 19.8571   643 14217.1429 22.1106 42.2857   643 26204.2857 40.7532    2.0000   643  1980  3.0793 152.3420   643 114997.1189 178.8447               0.0645   643  45.0924 0.0701                0.2573   643 179.5978 0.2793                0.7662   643  533.2297 0.8293                163.4178   643 124567.0708 193.7279  49.1350   643  30158.5340  46.9029
C        15603.5000   642  9961723.0000 15516.7025 32.0000   642 19807.5714 30.8529 40.4286   642 24491.4286 38.1486    3.0000   642  2568  4.0000 237.7450   642 166591.1365 259.4878               0.1104   642  76.8255 0.1197                0.4399   642 305.9260 0.4765                1.3097   642  907.9227 1.4142                250.5788   642 177682.3690 276.7638 123.6247   642  79632.4438 124.0381
B        15629.0000   642 10019841.0000 15607.2290 31.7857   642 19287.4286 30.0427 38.5714   642 22697.4286 35.3543    5.0000   642  3519  5.4813 337.5706   642 226305.2554 352.5004               0.1464   642 103.1852 0.1607                0.5832   642 410.7112 0.6397                1.7283   642 1217.8046 1.8969                352.2338   642 238145.9056 370.9438 217.1270   642 142717.9354 222.3021
A        15296.0000   643  9891223.0000 15382.9285 35.5714   643 20059.7143 31.1971 39.7143   643 22074.8571 34.3310    7.0000   643  7229 11.2426 479.5050   643 376564.6368 585.6371               0.2305   643 174.6416 0.2716                0.9126   643 695.2606 1.0813                2.6914   643 2062.4871 3.2076                499.4129   643 391021.7781 608.1210 443.8187   643 427622.3743 665.0426

In [39]:
cltv_final_eng12.groupby("segment").agg({"count", "mean","median", "sum"})

Customer ID                                recency                                T                          frequency                     monetary                            expected_purc_1_week                       expected_purc_1_month                       expected_purc_3_month                        expected_average_profit                                  clv                             
             median count           sum       mean  median count        sum    mean  median count        sum    mean    median count   sum    mean   median count         sum     mean               median count      sum   mean                median count      sum   mean                median count       sum   mean                  median count         sum     mean    median count          sum      mean
segment                                                                                                                                                                                                                                                                                                                                                                                                             
D        15756.0000   643 10097639.0000 15703.9487 19.5714   643 14117.5714 21.9558 42.2857   643 26067.4286 40.5403    2.0000   643  1965  3.0560 151.9600   643 114781.6046 178.5095               0.0645   643  45.2791 0.0704                0.2573   643 180.3307 0.2805                0.7662   643  535.3380 0.8326                162.8535   643 124376.0382 193.4309  535.1071   643  327543.3770  509.3987
C        15599.0000   642  9961716.0000 15516.6916 32.0000   642 19781.7143 30.8126 40.4286   642 24495.2857 38.1547    3.0000   642  2564  3.9938 237.7450   642 166521.5478 259.3794               0.1111   642  76.6941 0.1195                0.4428   642 305.4060 0.4757                1.3158   642  906.3967 1.4118                250.5788   642 177626.6076 276.6770 1340.4841   642  863835.3374 1345.5379
B        15626.5000   642 10020495.0000 15608.2477 31.5714   642 19202.2857 29.9101 38.5714   642 22604.1429 35.2089    5.0000   642  3504  5.4579 336.8257   642 225625.3018 351.4413               0.1473   642 103.3897 0.1610                0.5856   642 411.5057 0.6410                1.7341   642 1220.0387 1.9004                351.0233   642 237494.1815 369.9286 2353.3010   642 1542821.8934 2403.1494
A        15298.0000   643  9893122.0000 15385.8818 35.7143   643 20270.2857 31.5246 40.0000   643 22301.1429 34.6830    7.0000   643  7263 11.2955 481.3556   643 377529.6933 587.1379               0.2288   643 174.3817 0.2712                0.9071   643 694.2532 1.0797                2.6754   643 2059.6707 3.2032                500.6245   643 391920.2961 609.5183 4808.1116   643 4628403.1056 7198.1386

Hope it will be useful :)

linkedin.com/in/demir-zumrut/